## Import all Libraries

In [1]:
# AWS Rekognition to get bbox
import numpy as np
import boto3
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont
from matplotlib import pyplot as plt
from utils.rekognition import determine_color, draw_animal_count
import cv2
import time
import math
import os
from utils.config import *
from utils.fix_annotation import *

In [2]:
# process whole image.py to get key points
import mmcv
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint
import torch as tr
#from torchvision import transforms
from mmpose.apis import (inference, inference_top_down_pose_model, init_pose_model,
                         vis_pose_result)
from mmpose.models import build_posenet
from mmpose.datasets.pipelines import Compose

FNT = ImageFont.truetype('/usr/share/fonts/default/Type1/n019004l.pfb', 25)


## Get Bounding Boxes from Video Frames

In [3]:
class LoadImage:
    """A simple pipeline to load image."""

    def __init__(self, color_type='color', channel_order='rgb'):
        self.color_type = color_type
        self.channel_order = channel_order

    def __call__(self, results):
        """Call function to load images into results.
        Args:
            results (dict): A result dict contains the img_or_path.
        Returns:
            dict: ``results`` will be returned containing loaded image.
        """
        if isinstance(results['img_or_path'], str):
            results['image_file'] = results['img_or_path']
            img = mmcv.imread(results['img_or_path'], self.color_type,
                              self.channel_order)
        elif isinstance(results['img_or_path'], np.ndarray):
            results['image_file'] = ''
            if self.color_type == 'color' and self.channel_order == 'rgb':
                img = cv2.cvtColor(results['img_or_path'], cv2.COLOR_BGR2RGB)
        else:
            raise TypeError('"img_or_path" must be a numpy array or a str or '
                            'a pathlib.Path object')
        results['img'] = img
        return results

In [4]:
def init_pose_model(config, checkpoint=None, device='cuda:0'):
    """Initialize a pose model from config file.
    Args:
        config (str or :obj:`mmcv.Config`): Config file path or the config
            object.
        checkpoint (str, optional): Checkpoint path. If left as None, the model
            will not load any weights.
    Returns:
        nn.Module: The constructed detector.
    """
    if isinstance(config, str):
        config = mmcv.Config.fromfile(config)
    elif not isinstance(config, mmcv.Config):
        raise TypeError('config must be a filename or Config object, '
                        f'but got {type(config)}')
    config.model.pretrained = None
    model = build_posenet(config.model)
    if checkpoint is not None:
        # load model checkpoint
        load_checkpoint(model, checkpoint, map_location=device)
    # save the config in the model for convenience
    model.cfg = config
    model.to(device)
    model.eval()
    return model

In [5]:
def _box2cs(cfg, box):
    """This encodes bbox(x,y,w,h) into (center, scale)
    Args:
        x, y, w, h
    Returns:
        tuple: A tuple containing center and scale.
        - np.ndarray[float32](2,): Center of the bbox (x, y).
        - np.ndarray[float32](2,): Scale of the bbox w & h.
    """

    x, y, w, h = box[:4]
    input_size = cfg.data_cfg['image_size']
    aspect_ratio = input_size[0] / input_size[1]
    center = np.array([x + w * 0.5, y + h * 0.5], dtype=np.float32)

    if w > aspect_ratio * h:
        h = w * 1.0 / aspect_ratio
    elif w < aspect_ratio * h:
        w = h * aspect_ratio

    # pixel std is 200.0
    scale = np.array([w / 200.0, h / 200.0], dtype=np.float32)

    scale = scale * 1.25

    return center, scale

In [6]:
def process_model(model, dataset, person_results, img_or_path):
    bboxes = np.array([box['bbox'] for box in person_results])
    cfg = model.cfg
    flip_pairs = None
    device = next(model.parameters()).device
    channel_order = cfg.test_pipeline[0].get('channel_order', 'rgb')
    test_pipeline = [LoadImage(channel_order=channel_order)] + cfg.test_pipeline[1:]
    test_pipeline = Compose(test_pipeline)
    if dataset == 'AnimalHorse10Dataset':
        flip_pairs = []
    else:
        raise NotImplementedError()
    batch_data = []
    for bbox in bboxes:
        center, scale = _box2cs(cfg, bbox)
        # prepare data
        data = {
            'img_or_path':
            img_or_path,
            'center':
            center,
            'scale':
            scale,
            'bbox_score':
            bbox[4] if len(bbox) == 5 else 1,
            'bbox_id':
            0,  # need to be assigned if batch_size > 1
            'dataset':
            dataset,
            'joints_3d':
            np.zeros((cfg.data_cfg.num_joints, 3), dtype=np.float32),
            'joints_3d_visible':
            np.zeros((cfg.data_cfg.num_joints, 3), dtype=np.float32),
            'rotation':
            0,
            'ann_info': {
                'image_size': np.array(cfg.data_cfg['image_size']),
                'num_joints': cfg.data_cfg['num_joints'],
                'flip_pairs': flip_pairs
            }
        }
        data = test_pipeline(data)
        #print(data)
        batch_data.append(data)
    batch_data = collate(batch_data, samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter not work so just move image to cuda device
        batch_data['img'] = batch_data['img'].to(device)
    # get all img_metas of each bounding box
    batch_data['img_metas'] = [
        img_metas[0] for img_metas in batch_data['img_metas'].data
    ]

    with tr.no_grad():
        result = model(
            img=batch_data['img'],
            #img = torch_data,
            img_metas=batch_data['img_metas'],
            return_loss=False,
            return_heatmap=False)
    return result['preds'], result['output_heatmap']

In [7]:
def get_kp_color(label):
    # BGR
    color = (0, 0, 255)
    if label == 'Head':
        color = (51, 153, 51242)
    elif label == 'Spine':
        color = (253, 149, 66)
    elif label == 'Tail':
        color = (251, 82, 238)
    return color

In [8]:
def vis_pose(img, points, draw, label):
    points = points[0]
    if label == 'Tail':
        print(points)
    CS_THR = 0.3
    kp_color = get_kp_color(label)
    for i, point in enumerate(points):
        x, y, p = point
        if p > CS_THR:
            x = int(x)
            y = int(y)
            draw.ellipse([(x-10, y-10), (x+10, y+10)], fill=kp_color, outline=None)
            draw.text((x-10, y-30), str(i), font=FNT, fill=(255, 255, 255))
    return draw

In [9]:
device = tr.device("cuda:0" if tr.cuda.is_available() else "cpu")


model_head = init_pose_model(config='myConfigs/train_head_resnet.py', checkpoint='temp_logs/cattle_head/resnet/best.pth', device = device)
model_spine = init_pose_model(config='myConfigs/train_spine_resnet.py', checkpoint='temp_logs/cattle_spine/resnet/best.pth', device = device)
model_tail = init_pose_model(config='myConfigs/train_tail_hrnet.py', checkpoint='temp_logs/cattle_tail/hrnet/best.pth', device = device)

dataset_head = model_head.cfg.data['test']['type']
dataset_spine = model_spine.cfg.data['test']['type']
dataset_tail = model_tail.cfg.data['test']['type']

Use load_from_local loader
Use load_from_local loader
Use load_from_local loader


In [10]:
tail_count = 0
#draw response
def draw_response(frame,response, animal_target):
    global tail_count
    tail_check = False
    original_image = Image.fromarray(frame).copy()

    #draw the target count here first
    #image = draw_animal_count(response, animal_target, original_image)
    image = original_image
    # original image size
    imgWidth, imgHeight = image.size
    draw = ImageDraw.Draw(image)

    for customLabel in response['CustomLabels']:

        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']
            
            #print(customLabel['Name'])
            label = customLabel['Name']
        
            text = label
            text_width, text_height = FNT.getsize(label)
            color = determine_color(label, True)

            button_width = int(text_width + 20)
            button_height = int(text_height + 15)
            button_size = (button_width, button_height)
            button_img = Image.new('RGB', button_size, color)
            button_draw = ImageDraw.Draw(button_img)
            button_draw.text((10, 10), text, fill ='#000000', font=FNT)
            image.paste(button_img, (int(left), int(top)))  

#***** Keypoints
            if label == 'Head':
                np_image = np.array(image)
                head_bbox = [left, top, width, height]
                head_result = []
                head_result.append({'bbox': head_bbox})
                preds, _ = process_model(model_head, dataset_head, head_result, np_image)
                draw = vis_pose(np_image, preds, draw, 'Head')
            elif label == 'Cow':
                np_image = np.array(image)
                cow_bbox = [left, top, width, height]
                cow_result = []
                cow_result.append({'bbox': cow_bbox})
                preds, _ = process_model(model_spine, dataset_spine, cow_result, np_image)
                draw = vis_pose(np_image, preds, draw, 'Spine')
            elif label == 'Tail':
                np_image = np.array(image)
                tail_bbox = [left, top, width, height]
                tail_result = []
                tail_result.append({'bbox': tail_bbox})
                preds, _ = process_model(model_tail, dataset_tail, tail_result, np_image)
                draw = vis_pose(np_image, preds, draw, 'Tail')
                tail_check = True
#*****
            points = (
                (left, top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            
            thickness = 5
            
            if label == 'cow':
                thickness = 7
                
            draw.line(points, fill=color, width=thickness)
    
    img = np.asarray(image)[:,:,::-1].copy()
    # check tail label
    inferred_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if tail_check:
        print('Tail')
        plt.imshow(inferred_frame)
        plt.title("Tail Image {}".format(tail_count))
        #plt.show()
        plt.savefig('frame_imgs/tail_{}.png'.format(tail_count), dpi=100)
        tail_count+=1
    return img

In [11]:
def analyzeVideo(src_file, output_file, projectVersionArn, fps=5, min_confidence=75):
    
    start = time.time()
    
    rekognition = boto3.client('rekognition')        
    cap = cv2.VideoCapture(src_file)
    frameRate = cap.get(fps) #frame rate
    
    #function to write a video
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    
    imgSize = (int(width), int(height))
    
    #fourcc = cv2.VideoWriter_fourcc(*'XVID')
    #writer = cv2.VideoWriter(output_file, fourcc, frameRate, imgSize, True)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(output_file, fourcc, frameRate, imgSize)
    
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        
        ret, frame = cap.read()
       
        if (ret != True):
            break
        else:
            #inference on the extracted frame
            hasFrame, imageBytes = cv2.imencode(".jpg", frame)

            if(hasFrame):
                response = rekognition.detect_custom_labels(
                    Image={
                        'Bytes': imageBytes.tobytes(),
                    },
                    MinConfidence=min_confidence,
                    ProjectVersionArn = projectVersionArn
                )     

            inferred_frame = draw_response(frame, response, animal_target='cow')
            inferred_frame = cv2.cvtColor(inferred_frame, cv2.COLOR_BGR2RGB)
            if frameId % 25 == 0:
                print("Finish Processing {} frame".format(frameId))
                plt.imshow(inferred_frame)
                plt.title("Frame {}".format(int(frameId)))
                #plt.show()
                
                plt.savefig('frame_imgs/frame{}.png'.format(int(frameId)), dpi=150)
                lap = time.time()
                print('lap time: ', lap - start)
            videoWriter.write(inferred_frame)

    cap.release()
    videoWriter.release()
    cv2.destroyAllWindows()
    
    end = time.time()
    print('total time lapse', end - start)

In [12]:
six_class_arn = 'arn:aws:rekognition:us-east-1:617989056061:project/cow-detector/version/cow-detector.2021-05-11T13.23.57/1620764637425'
src_video = 'data/demo_videos/cattle_single_2.mov'
output_video = 'output.mp4'

analyzeVideo(src_video, output_video, six_class_arn)
print('finished analyzing the video')

Finish Processing 0.0 frame
lap time:  3.447202444076538
Finish Processing 25.0 frame
lap time:  68.09503507614136
[[1.3161713e+03 1.1333043e+02 5.3645510e-02]
 [1.2842981e+03 1.0650047e+02 7.1195185e-02]
 [1.3548743e+03 1.8845982e+02 3.6390401e-02]
 [1.3890239e+03 2.4765266e+02 1.6857785e-01]
 [1.4641533e+03 3.2505872e+02 7.3237374e-02]]
Tail
Finish Processing 50.0 frame
lap time:  133.20639181137085
Finish Processing 75.0 frame
lap time:  197.70557403564453
Finish Processing 100.0 frame
lap time:  261.7056248188019
Finish Processing 125.0 frame
lap time:  324.18029284477234
Finish Processing 150.0 frame
lap time:  387.99613785743713
Finish Processing 175.0 frame
lap time:  451.1653504371643
Finish Processing 200.0 frame
lap time:  517.8042144775391
Finish Processing 225.0 frame
lap time:  583.460033416748
Finish Processing 250.0 frame
lap time:  650.1343998908997
Finish Processing 275.0 frame
lap time:  717.6574511528015
Finish Processing 300.0 frame
lap time:  785.9042270183563
Fini

[[1.5870468e+03 1.5411679e+02 8.9692563e-02]
 [1.6525430e+03 2.3926169e+02 4.3051201e-01]
 [1.6656421e+03 3.7025394e+02 3.5843557e-01]
 [1.6590925e+03 5.2744446e+02 3.3447623e-01]
 [1.7507869e+03 7.5668079e+02 3.0236357e-01]]
Tail
[[1.5896650e+03 1.3319785e+02 7.0878744e-02]
 [1.6558857e+03 2.2590677e+02 4.6603990e-01]
 [1.6625078e+03 3.7159225e+02 3.8927647e-01]
 [1.6625078e+03 5.3714392e+02 3.5337985e-01]
 [1.7552168e+03 7.5567212e+02 2.6619458e-01]]
Tail
[[1.57810144e+03 1.18284546e+02 1.00423291e-01]
 [1.65575391e+03 2.34763184e+02 4.43727076e-01]
 [1.66222485e+03 3.64183838e+02 4.27390009e-01]
 [1.66222485e+03 5.25959717e+02 3.48627746e-01]
 [1.73987744e+03 7.45974854e+02 1.86664030e-01]]
Tail
[[1.5713287e+03 1.2968442e+02 1.0775119e-01]
 [1.6561145e+03 2.1447006e+02 4.3416691e-01]
 [1.6561145e+03 3.5795358e+02 3.7853265e-01]
 [1.6561145e+03 5.2752502e+02 3.7258756e-01]
 [1.6691584e+03 7.2970630e+02 2.6615530e-01]]
Tail
Finish Processing 475.0 frame
lap time:  1490.6229693889618
[

[[1.4978904e+03 6.7030090e+01 1.8048534e-01]
 [1.5730422e+03 1.8727289e+02 4.1906071e-01]
 [1.5805574e+03 3.3757648e+02 4.0480527e-01]
 [1.5730422e+03 5.3297101e+02 4.0693456e-01]
 [1.5880728e+03 7.8097186e+02 3.6822754e-01]]
Tail
Finish Processing 525.0 frame
lap time:  2070.911500453949
[[1.5043003e+03 6.5296448e+01 1.8782040e-01]
 [1.5799226e+03 1.8629187e+02 4.3839744e-01]
 [1.5874846e+03 3.5266058e+02 4.2146087e-01]
 [1.5799226e+03 5.4927814e+02 4.3355778e-01]
 [1.5799226e+03 7.8370685e+02 4.6324301e-01]]
Tail
[[1.5022479e+03 6.0429504e+01 1.5260005e-01]
 [1.5766024e+03 1.8683203e+02 4.4853231e-01]
 [1.5840377e+03 3.5041174e+02 4.5169955e-01]
 [1.5766024e+03 5.4373328e+02 4.3218786e-01]
 [1.5542960e+03 7.8166736e+02 4.4437972e-01]]
Tail
[[1.4879407e+03 7.6203217e+01 1.7125270e-01]
 [1.5770331e+03 1.8014438e+02 4.5283568e-01]
 [1.5770331e+03 3.5090482e+02 4.5267779e-01]
 [1.5770331e+03 5.2908960e+02 4.2082429e-01]
 [1.5473356e+03 7.7409375e+02 5.1546425e-01]]
Tail
[[1.4834381e+03 7

[[1.2053417e+03 8.2780701e+01 2.6237530e-01]
 [1.2995951e+03 2.2844507e+02 4.6169466e-01]
 [1.3081636e+03 4.1695190e+02 3.3268052e-01]
 [1.3338691e+03 6.3116418e+02 4.0592057e-01]
 [1.3338691e+03 9.0535596e+02 5.5090833e-01]]
Tail
[[1.2186499e+03 8.5749756e+01 2.2598515e-01]
 [1.2954749e+03 2.3086365e+02 4.7771859e-01]
 [1.2954749e+03 4.1865820e+02 3.5149482e-01]
 [1.3210833e+03 6.2352490e+02 3.8747978e-01]
 [1.3296194e+03 9.0521667e+02 5.8663684e-01]]
Tail
[[1.2072451e+03 8.7304932e+01 2.9476336e-01]
 [1.2840701e+03 2.3241882e+02 4.2958364e-01]
 [1.2926062e+03 4.1167725e+02 3.3087072e-01]
 [1.3096785e+03 6.3361621e+02 3.4555635e-01]
 [1.3267507e+03 9.0677185e+02 6.0674584e-01]]
Tail
[[1.2028252e+03 8.6991760e+01 3.2615209e-01]
 [1.2700957e+03 2.3835040e+02 4.0205687e-01]
 [1.2785044e+03 4.2334430e+02 3.1933573e-01]
 [1.2953220e+03 6.4197339e+02 3.5016924e-01]
 [1.3289573e+03 8.9423779e+02 4.7823781e-01]]
Tail
[[1.2032983e+03 8.0160706e+01 2.9411298e-01]
 [1.2616123e+03 2.3844128e+02 4

Tail
[[8.19601074e+02 1.98249512e+02 3.84300470e-01]
 [6.01626099e+02 2.06968506e+02 1.26266614e-01]
 [8.71915039e+02 5.47009277e+02 1.08045205e-01]
 [8.45758057e+02 7.82422241e+02 1.62522137e-01]
 [7.41130127e+02 1.00039709e+03 1.74821317e-01]]
Tail
[[8.3157593e+02 1.9474976e+02 3.6099017e-01]
 [8.6694214e+02 3.8926392e+02 1.6865425e-01]
 [8.8462524e+02 5.6609485e+02 1.3718387e-01]
 [8.4925903e+02 7.7829211e+02 2.1416025e-01]
 [3.4529065e+02 9.2859851e+02 1.7280452e-01]]
Tail
[[8.3529657e+02 2.0245392e+02 4.6233603e-01]
 [8.6900140e+02 3.8783063e+02 2.2082551e-01]
 [8.6900140e+02 5.9005988e+02 2.2027105e-01]
 [8.5214899e+02 7.6701031e+02 2.4734420e-01]
 [3.7185474e+02 9.1868219e+02 2.0043540e-01]]
Tail
[[8.5078790e+02 1.9046686e+02 3.8360924e-01]
 [8.8495514e+02 3.8692847e+02 1.6195628e-01]
 [8.8495514e+02 5.8339008e+02 1.7743513e-01]
 [8.5932965e+02 7.6276801e+02 2.0606539e-01]
 [7.2266083e+02 1.0275640e+03 1.8666539e-01]]
Tail
[[8.5858942e+02 1.8573798e+02 3.4784570e-01]
 [8.8538495

...

Tail
[[4.6699371e+02 2.0639514e+02 4.2362607e-01]
 [4.1244360e+02 3.6225262e+02 2.3364787e-01]
 [4.0465076e+02 5.4928156e+02 2.0515604e-01]
 [3.8127216e+02 7.2851776e+02 3.0289465e-01]
 [3.5010065e+02 9.4671820e+02 2.0917040e-01]]
[[1.3848871e+02 1.0469706e+02 5.9195451e-02]
 [1.6626947e+02 1.5348045e+01 5.7814687e-02]
 [1.9179776e+02 1.3698282e+02 7.3091671e-02]
 [2.0381110e+02 1.7902942e+02 2.0282234e-01]
 [2.7138596e+02 1.8503607e+02 6.7749575e-02]]
Tail
[[4.6183685e+02 2.2345111e+02 3.7407604e-01]
 [4.0072827e+02 3.7622260e+02 2.7487087e-01]
 [3.9308966e+02 5.4427124e+02 2.1254131e-01]
 [3.8545111e+02 7.2759705e+02 3.1410718e-01]
 [1.1046240e+02 9.5675427e+02 1.5830195e-01]]
[[1.5122070e+02 7.3340263e+01 3.8738858e-02]
 [2.1526630e+02 2.5126167e+01 4.0037476e-02]
 [2.0087402e+02 1.3450742e+02 9.0987153e-02]
 [2.1382706e+02 1.6760962e+02 2.0359097e-01]
 [3.0305911e+02 1.7336655e+02 5.8415867e-02]]
Tail
[[4.4806519e+02 2.3146716e+02 2.9905057e-01]
 [3.8837579e+02 3.7322949e+02 2.8218

[[3.8760760e+02 1.3146228e+02 4.7809821e-02]
 [3.9213867e+02 1.1786914e+02 7.4162111e-02]
 [2.8339362e+02 3.3989026e+02 1.5372497e-01]
 [2.9698676e+02 4.4863531e+02 1.7352226e-01]
 [3.6495245e+02 6.4800116e+02 4.9154535e-02]]
Tail
[[3.7398416e+02 1.3717197e+02 4.9030103e-02]
 [3.8767093e+02 1.2348520e+02 6.6290572e-02]
 [2.7361441e+02 3.5159830e+02 1.7694344e-01]
 [2.8730118e+02 5.1127744e+02 2.1044873e-01]
 [5.9525391e+02 6.2305298e+02 1.0310358e-01]]
Tail
[[3.9373221e+02 1.4000562e+02 4.6173804e-02]
 [5.8996216e+01 1.7396432e+02 8.3724678e-02]
 [2.6274857e+02 3.5831174e+02 1.3308057e-01]
 [2.7730234e+02 4.9899783e+02 2.6354834e-01]
 [2.4819479e+02 6.4453528e+02 7.0326120e-02]]
Tail
[[1.0167963e+02 1.1898227e+02 6.4774886e-02]
 [4.5032336e+02 7.1630908e+02 8.6241052e-02]
 [2.4552563e+02 3.4084650e+02 7.7677779e-02]
 [2.6990637e+02 4.7737830e+02 2.5077337e-01]
 [4.6495178e+02 6.9192834e+02 8.4785007e-02]]
Tail
[[9.1323364e+01 1.1979764e+02 7.9790406e-02]
 [2.6081635e+02 6.9705627e+02 7

[[1.2384568e+03 2.6753415e+02 4.5823857e-02]
 [1.2820652e+03 1.6668979e+02 7.2878972e-02]
 [1.2527657e+03 2.6685278e+02 1.8244424e-01]
 [1.2841093e+03 2.7366656e+02 1.5145612e-01]
 [1.2827466e+03 2.6276447e+02 6.7740969e-02]]
Tail
[[1.3448138e+03 1.9827515e+02 3.2115974e-02]
 [1.2735718e+03 1.9964520e+02 5.1163398e-02]
 [1.2441160e+03 2.9349286e+02 5.4184958e-02]
 [1.2612415e+03 1.9827515e+02 1.2245028e-01]
 [1.3379636e+03 3.5171954e+02 6.0517218e-02]]
Tail
[[1.3419972e+03 2.3912668e+02 4.2640172e-02]
 [1.2656517e+03 2.2784085e+02 6.8487242e-02]
 [1.2663157e+03 2.3381570e+02 4.9355280e-02]
 [1.3738632e+03 3.3737991e+02 1.2853497e-01]
 [1.3964348e+03 3.8385101e+02 6.0592905e-02]]
Tail
[[1.2299685e+03 3.1508130e+02 5.6800425e-02]
 [1.2629203e+03 2.4917767e+02 6.1207913e-02]
 [1.3981600e+03 3.1576779e+02 4.1269679e-02]
 [1.2491904e+03 2.4917767e+02 1.0525094e-01]
 [1.3967870e+03 4.0913126e+02 6.0302559e-02]]
Tail
[[1.2282579e+03 3.1549133e+02 4.1719504e-02]
 [1.2760774e+03 2.6767194e+02 5

[[1.5588163e+03 2.0275244e+02 3.9617307e-02]
 [1.4071445e+03 2.1113632e+02 5.3531855e-02]
 [1.5473838e+03 2.7820728e+02 6.6288665e-02]
 [1.3919011e+03 2.1113632e+02 9.8456681e-02]
 [1.5031779e+03 3.8795972e+02 8.6075500e-02]]
Tail
[[1.4890011e+03 2.1831679e+02 4.7918465e-02]
 [1.4127473e+03 2.0905232e+02 6.7236647e-02]
 [1.5417373e+03 2.8245551e+02 4.2690583e-02]
 [1.4604950e+03 2.1831679e+02 8.6970441e-02]
 [1.5075300e+03 3.7937622e+02 8.8858299e-02]]
Tail
[[1.3922263e+03 1.9286908e+02 3.9632201e-02]
 [1.4301921e+03 2.0821701e+02 5.1686663e-02]
 [1.3922263e+03 1.9286908e+02 4.5569293e-02]
 [1.5384354e+03 3.2292252e+02 1.1168811e-01]
 [1.3881874e+03 3.8269864e+02 9.1171294e-02]]
Tail
[[1.4083173e+03 2.0302921e+02 4.2157862e-02]
 [1.5365420e+03 2.4525888e+02 6.4883560e-02]
 [1.5457557e+03 2.7904263e+02 6.2314548e-02]
 [1.5442201e+03 3.2664703e+02 9.5965475e-02]
 [1.5135076e+03 3.8192953e+02 1.0838983e-01]]
Tail
[[1.4786974e+03 2.1771939e+02 4.3215156e-02]
 [1.4312883e+03 2.0823761e+02 6

[[1.3133678e+03 2.3746283e+02 6.4536750e-02]
 [1.4324259e+03 1.9344739e+02 7.4338853e-02]
 [1.4771630e+03 2.5983133e+02 5.2325591e-02]
 [1.3710929e+03 1.8262393e+02 7.8364484e-02]
 [1.4814923e+03 3.5363470e+02 5.8142267e-02]]
Tail
[[1.3084781e+03 1.5739380e+02 5.0579298e-02]
 [1.3636436e+03 1.6401364e+02 5.8547739e-02]
 [1.4725033e+03 2.3609647e+02 4.8580304e-02]
 [1.4680901e+03 2.5816266e+02 7.7054076e-02]
 [1.3651146e+03 1.9931952e+02 4.9770556e-02]]
[[1.1340155e+03 1.4995645e+02 3.1408571e-02]
 [1.1075946e+03 1.4827000e+02 6.8981752e-02]
 [1.0654335e+03 1.9717693e+02 5.2252255e-02]
 [1.0643091e+03 2.4327309e+02 2.3472857e-01]
 [1.0541904e+03 2.7812628e+02 7.1073741e-02]]
Tail
[[1.30411755e+03 1.37831207e+02 4.44957688e-02]
 [1.35525903e+03 1.43019470e+02 5.99492602e-02]
 [1.46643640e+03 2.11208191e+02 4.69336286e-02]
 [1.33895312e+03 2.54196716e+02 1.03066385e-01]
 [1.42048303e+03 3.12008881e+02 1.01214871e-01]]
[[1.0404391e+03 2.6861017e+02 4.4093605e-02]
 [1.0505228e+03 1.2328690e

In [14]:
def save_frames(src_file, output_path, output_video, fps=5):
    start = time.time()
    
    cap = cv2.VideoCapture(src_file)
    frameRate = cap.get(fps) #frame rate
    
    #function to write a video
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    
    imgSize = (int(width), int(height))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(output_video, fourcc, frameRate, imgSize)
    
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        
        ret, frame = cap.read()
       
        if (ret != True):
            break
        else:
            #inference on the extracted frame
            hasFrame, imageBytes = cv2.imencode(".jpg", frame)

            if(hasFrame):
                # creating image object of array
                data = Image.fromarray(frame)

                # saving a JPG file
                data.save(output_path+'{}.jpg'.format(int(frameId)))
                
            if frameId % 50 == 0:
                print("Finish Processing {} frame".format(int(frameId)))
                lap = time.time()
                print('lap time: ', lap - start)
            videoWriter.write(frame)

    cap.release()
    videoWriter.release()
    cv2.destroyAllWindows()
    
    end = time.time()
    print('total time lapse', end - start)

src_video = 'data/demo_videos/cattle_single_2.mov'
output_path = 'frame_imgs/cattle_single_2/'
output_video = 'frame_imgs/cattle_single_2/ori_video.mp4'
save_frames(src_video, output_path, output_video)

Finish Processing 0 frame
lap time:  0.21416759490966797
Finish Processing 50 frame
lap time:  7.817583322525024
Finish Processing 100 frame
lap time:  15.077166318893433
Finish Processing 150 frame
lap time:  22.338428735733032
Finish Processing 200 frame
lap time:  29.654836416244507
Finish Processing 250 frame
lap time:  36.72226691246033
Finish Processing 300 frame
lap time:  44.16100311279297
Finish Processing 350 frame
lap time:  51.364588499069214
Finish Processing 400 frame
lap time:  58.42804026603699
Finish Processing 450 frame
lap time:  65.80324268341064
Finish Processing 500 frame
lap time:  73.16969919204712
Finish Processing 550 frame
lap time:  80.52755856513977
Finish Processing 600 frame
lap time:  87.83010292053223
Finish Processing 650 frame
lap time:  95.23773193359375
Finish Processing 700 frame
lap time:  102.60534334182739
Finish Processing 750 frame
lap time:  110.26758861541748
Finish Processing 800 frame
lap time:  117.27085208892822
Finish Processing 850 fra

In [13]:
# %%HTML
# <video width="400" height="320" controls>
#   <source src="bbox_out.mov" type="video/mov">
# </video